<a href="https://colab.research.google.com/github/Rakeshkrishnamurthy/Rocky_Help/blob/main/Telecom24_Project_Docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***1) First creating of schema and parameters tables in hive metastore***

***metadata_table_creation_scrept***

In [ ]:
#Creating parameter for Storage account
dbutils.widgets.text("storage_account_name", "")
storage_account_name = dbutils.widgets.get("storage_account_name")

#Creating SQL DB for Schema
%sql
create database if not exists hive_metastore.metadata_schema

#tbl_source_control metadata table holds the parameters values for various connections
spark.sql(f"""
create or replace table metadata_schema.tbl_source_control (

    gcp_source_bucket string,
    storage_account string,
    adls_url string,
    container_name string,
    logic_app_url string,
    email_id string
)
location 'abfss://metadata@{storage_account_name}.dfs.core.windows.net/tbl_source_control'
""")

# tbl_parameters holds the parameter values of DDL
spark.sql(f"""
create or replace table metadata_schema.tbl_parameters (

    job_id int,
    source_file_path string,
    watermark_column timestamp,
    raw_schema string,
    raw_tbl string,
    intermediate_schema string,
    intermediate_tbl string,
    intermediate_query string,
    curated_schema string,
    curated_tbl string,
    curated_query string
)
location 'abfss://metadata@{storage_account_name}.dfs.core.windows.net/tbl_parameters'

***raw_tbl_creation***

In [ ]:
#parameting for getting environment and storage account for RAW LAYER
dbutils.widgets.text("storage_account_name", "")
storage_account_name = dbutils.widgets.get("storage_account_name")

dbutils.widgets.text("env", "")
env = dbutils.widgets.get("env")

#Creating the data base for raw
spark.sql(f"create database if not exists hive_metastore.{env}_raw")

#creating dimention table of CITIES in raw layer
spark.sql(f"""
create or replace table {env}_raw.dim_cities (

    city_code int,
    city_name string,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/dim_cities'
""")

#creating dimention table of date in raw layer
spark.sql(f"""
create or replace table {env}_raw.dim_date (

    dated string,
    month_name string,
    before_or_after_5g string,
    time_period int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/dim_date'
""")

#creating dimention table of plan in raw layer
spark.sql(f"""
create or replace table {env}_raw.dim_plan (

    plan string,
    plan_description string,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/dim_plan'
""")


#Fact Tables for metrics_share
spark.sql(f"""
create or replace table {env}_raw.fact_metrics_share (

    dated string,
    city_code int,
    company string,
    company_revenue_crores double,
    arpu int,
    active_users_lakhs double,
    unsubscribed_users_lakhs double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/fact_metrics_share'
""")

#Fact Table for Market_share
spark.sql(f"""
create or replace table {env}_raw.fact_market_share (

    dated string,
    city_code int,
    tmv_city_crores double,
    company string,
    ms_pct double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/fact_market_share'
""")

#Fact table plan_revenue
spark.sql(f"""
create or replace table {env}_raw.fact_plan_revenue (

    dated string,
    city_code int,
    plan string,
    plan_revenue_crores double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://raw@{storage_account_name}.dfs.core.windows.net/fact_plan_revenue'
""")

***log_table_creation_screpts***

In [ ]:
#parameting for getting environment and storage account
dbutils.widgets.text("storage_account_name", "")
storage_account_name = dbutils.widgets.get("storage_account_name")
dbutils.widgets.text("env", "")
env = dbutils.widgets.get("env")

#Creating the data base for raw
spark.sql(f"""
create database if not exists hive_metastore.{env}_log
""")

#Log recode table
spark.sql(f"""
create or replace table {env}_log.log_record_tbl (
    env string,
    pipeLineName string,
    logMessage string,
    status string,
    triggerType string,
    loadId string,
    logTimeStamp timestamp
)
location 'abfss://log@{storage_account_name}.dfs.core.windows.net/log_record_tbl'
""")

***intermediate_tables_creation_scripts***

In [ ]:
#parameting for getting environment and storage account for INTERMEDIATE
dbutils.widgets.text("storage_account_name", "")
storage_account_name = dbutils.widgets.get("storage_account_name")

dbutils.widgets.text("env", "")
env = dbutils.widgets.get("env")

#Creating Sql DB
spark.sql(f"create database if not exists hive_metastore.{env}_intermediate")

#creating dimention table of cities in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.dim_cities (

    city_code int,
    city_name string,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/dim_cities'
""")

#creating dimention table of date in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.dim_date (

    dated string,
    month_name string,
    before_or_after_5g string,
    time_period int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/dim_date'
""")

#creating dimention table of plan in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.dim_plan (

    plan string,
    plan_description string,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/dim_plan'
""")

#creating dimention table of metrics_share in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.fact_metrics_share (

    dated string,
    city_code int,
    company string,
    company_revenue_crores double,
    arpu int,
    active_users_lakhs double,
    unsubscribed_users_lakhs double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/fact_metrics_share'
""")

#creating dimention table of market_share in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.fact_market_share (

    dated string,
    city_code int,
    tmv_city_crores double,
    company string,
    ms_pct double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/fact_market_share'
""")

#creating dimention table of plan_revenue in intermediate layer
spark.sql(f"""
create or replace table {env}_intermediate.fact_plan_revenue (

    dated string,
    city_code int,
    plan string,
    plan_revenue_crores double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://intermediate@{storage_account_name}.dfs.core.windows.net/fact_plan_revenue'
""")

**curated_tables_creation_scripts**

In [ ]:
#parameting for getting environment and storage account for CURATED
dbutils.widgets.text("storage_account_name", "")
storage_account_name = dbutils.widgets.get("storage_account_name")

dbutils.widgets.text("env", "")
env = dbutils.widgets.get("env")

#Creation of SQL DB
spark.sql(f"create database if not exists hive_metastore.{env}_curated")

#creating of table metrics_share in CURATED layer
spark.sql(f"""
create or replace table {env}_curated.metrics_share (

    before_or_after_5g string,
    city_name string,
    month_name string,
    time_period int,
    dated string,
    city_code int,
    company string,
    company_revenue_crores double,
    arpu int,
    active_users_lakhs double,
    unsubscribed_users_lakhs double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://curated@{storage_account_name}.dfs.core.windows.net/metrics_share'
""")

#creating of table metrics_share in CURATED layer
spark.sql(f"""
create or replace table {env}_curated.market_share (

    before_or_after_5g string,
    city_name string,
    month_name string,
    time_period int,
    dated string,
    city_code int,
    tmv_city_crores double,
    company string,
    ms_pct double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://curated@{storage_account_name}.dfs.core.windows.net/market_share'
""")

#creating of table metrics_share in CURATED layer
spark.sql(f"""
create or replace table {env}_curated.plan_revenue (

    before_or_after_5g string,
    city_name string,
    month_name string,
    time_period int,
    plan_description string,
    dated string,
    city_code int,
    plan string,
    plan_revenue_crores double,
    seq_no int,
    last_inserted_dttm_azure timestamp,
    last_updated_dttm_gcp string,
    load_id string
)
partitioned by (city_code)
location 'abfss://curated@{storage_account_name}.dfs.core.windows.net/plan_revenue'
""")

***2) Creation if insertion for all the 3 layers***

***metadata_tables_insert_scripts_till_raw***

In [ ]:
#Paramaeting the detils for data moment from Source to RAW
dbutils.widgets.text("logic_app_url", "")
logic_app_url = dbutils.widgets.get("logic_app_url")

dbutils.widgets.text("email_id", "")
email_id = dbutils.widgets.get("email_id")

dbutils.widgets.text("storage_account", "")
storage_account = dbutils.widgets.get("storage_account")

dbutils.widgets.text("adls_url", "")
adls_url = dbutils.widgets.get("adls_url")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")


#Insertion to the sorce table by passing paramaetrs
spark.sql(f"""
insert into metadata_schema.tbl_source_control values
('{gcp_bucket}','{storage_account}','{adls_url}','landing','{logic_app_url}','{email_id}')
""")

#initail inertion of data to the source
spark.sql("""
insert into metadata_schema.tbl_parameters values
(201,'telecom/dim_cities','2001-01-01T15:13:23.963Z','dev_raw','dim_cities','dev_intermediate','dim_cities',NULL,NULL,NULL,NULL),
(202,'telecom/dim_date','2001-01-01T15:13:23.963Z','dev_raw','dim_date','dev_intermediate','dim_date',NULL,NULL,NULL,NULL),
(203,'telecom/dim_plan','2001-01-01T15:13:23.963Z','dev_raw','dim_plan','dev_intermediate','dim_plan',NULL,NULL,NULL,NULL),
(204,'telecom/fact_company','2001-01-01T15:13:23.963Z','dev_raw','fact_metrics_share','dev_intermediate','fact_metrics_share',NULL,NULL,NULL,NULL),
(205,'telecom/fact_market','2001-01-01T15:13:23.963Z','dev_raw','fact_market_share','dev_intermediate','fact_market_share',NULL,NULL,NULL,NULL),
(206,'telecom/fact_plan','2001-01-01T15:13:23.963Z','dev_raw','fact_plan_revenue','dev_intermediate','fact_plan_revenue',NULL,NULL,NULL,NULL)
""")

metadata_tables_insert_scripts_till_intermediate

In [ ]:
#Paramaeting the detils for data moment from Source to Intermediate
dbutils.widgets.text("logic_app_url", "")
logic_app_url = dbutils.widgets.get("logic_app_url")

dbutils.widgets.text("email_id", "")
email_id = dbutils.widgets.get("email_id")

dbutils.widgets.text("storage_account", "")
storage_account = dbutils.widgets.get("storage_account")

dbutils.widgets.text("adls_url", "")
adls_url = dbutils.widgets.get("adls_url")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")

#Insertion to intermediet from the sorce table by passing parametrs
spark.sql(f"""

insert into metadata_schema.tbl_source_control values
('{gcp_bucket}','{storage_account}','{adls_url}','landing','{logic_app_url}','{email_id}')

""")

#Full load and delta or Incremental load to Intermidate
spark.sql("""

insert into metadata_schema.tbl_parameters values
(201,'telecom/dim_cities','2001-01-01T15:13:23.963Z','dev_raw','dim_cities','dev_intermediate','dim_cities',NULL,NULL,NULL,NULL),
(202,'telecom/dim_date','2001-01-01T15:13:23.963Z','dev_raw','dim_date','dev_intermediate','dim_date',NULL,NULL,NULL,NULL),
(203,'telecom/dim_plan','2001-01-01T15:13:23.963Z','dev_raw','dim_plan','dev_intermediate','dim_plan',NULL,NULL,NULL,NULL),
(204,'telecom/fact_company','2001-01-01T15:13:23.963Z','dev_raw','fact_metrics_share','dev_intermediate','fact_metrics_share','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.company = source.company,
      target.company_revenue_crores = source.company_revenue_crores,
      target.arpu = source.arpu,
      target.active_users_lakhs = source.active_users_lakhs,
      target.unsubscribed_users_lakhs = source.unsubscribed_users_lakhs,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.company,
      target.company_revenue_crores,
      target.arpu,
      target.active_users_lakhs,
      target.unsubscribed_users_lakhs,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.company,
      source.company_revenue_crores,
      source.arpu,
      source.active_users_lakhs,
      source.unsubscribed_users_lakhs,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL),
(205,'telecom/fact_market','2001-01-01T15:13:23.963Z','dev_raw','fact_market_share','dev_intermediate','fact_market_share','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.tmv_city_crores = source.tmv_city_crores,
      target.company = source.company,
      target.ms_pct = source.ms_pct,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.tmv_city_crores,
      target.company,
      target.ms_pct,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.tmv_city_crores,
      source.company,
      source.ms_pct,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL),
(206,'telecom/fact_plan','2001-01-01T15:13:23.963Z','dev_raw','fact_plan_revenue','dev_intermediate','fact_plan_revenue','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.plan = source.plan,
      target.plan_revenue_crores = source.plan_revenue_crores,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.plan,
      target.plan_revenue_crores,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.plan,
      source.plan_revenue_crores,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL)
""")

metadata_tables_insert_scripts_till_curated

In [ ]:
#Paramaeting the detils for data moment from Source to Intermediate
dbutils.widgets.text("logic_app_url", "")
logic_app_url = dbutils.widgets.get("logic_app_url")

dbutils.widgets.text("email_id", "")
email_id = dbutils.widgets.get("email_id")

dbutils.widgets.text("storage_account", "")
storage_account = dbutils.widgets.get("storage_account")

dbutils.widgets.text("adls_url", "")
adls_url = dbutils.widgets.get("adls_url")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")

dbutils.widgets.text("gcp_bucket","")
gcp_bucket = dbutils.widgets.get("gcp_bucket")

##Insertion to curated from the intermidate by passing parametrs
spark.sql(f"""

insert into metadata_schema.tbl_source_control values
('{gcp_bucket}','{storage_account}','{adls_url}','landing','{logic_app_url}','{email_id}')

""")

#Full load and incremental load from Intermidate to Curated
spark.sql("""

insert into metadata_schema.tbl_parameters values
(201,'telecom/dim_cities','2001-01-01T15:13:23.963Z','dev_raw','dim_cities','dev_intermediate','dim_cities',NULL,NULL,NULL,NULL),
(202,'telecom/dim_date','2001-01-01T15:13:23.963Z','dev_raw','dim_date','dev_intermediate','dim_date',NULL,NULL,NULL,NULL),
(203,'telecom/dim_plan','2001-01-01T15:13:23.963Z','dev_raw','dim_plan','dev_intermediate','dim_plan',NULL,NULL,NULL,NULL),
(204,'telecom/fact_company','2001-01-01T15:13:23.963Z','dev_raw','fact_metrics_share','dev_intermediate','fact_metrics_share','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.company = source.company,
      target.company_revenue_crores = source.company_revenue_crores,
      target.arpu = source.arpu,
      target.active_users_lakhs = source.active_users_lakhs,
      target.unsubscribed_users_lakhs = source.unsubscribed_users_lakhs,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.company,
      target.company_revenue_crores,
      target.arpu,
      target.active_users_lakhs,
      target.unsubscribed_users_lakhs,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.company,
      source.company_revenue_crores,
      source.arpu,
      source.active_users_lakhs,
      source.unsubscribed_users_lakhs,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL),
(205,'telecom/fact_market','2001-01-01T15:13:23.963Z','dev_raw','fact_market_share','dev_intermediate','fact_market_share','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.tmv_city_crores = source.tmv_city_crores,
      target.company = source.company,
      target.ms_pct = source.ms_pct,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.tmv_city_crores,
      target.company,
      target.ms_pct,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.tmv_city_crores,
      source.company,
      source.ms_pct,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL),
(206,'telecom/fact_plan','2001-01-01T15:13:23.963Z','dev_raw','fact_plan_revenue','dev_intermediate','fact_plan_revenue','merge into {intermediate_schema}.{intermediate_table} target
  using temp_view source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.plan = source.plan,
      target.plan_revenue_crores = source.plan_revenue_crores,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = source.load_id
  when not matched then
    insert (
      target.dated,
      target.city_code,
      target.plan,
      target.plan_revenue_crores,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.dated,
      source.city_code,
      source.plan,
      source.plan_revenue_crores,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      source.load_id
    )',NULL,NULL,NULL),
    (207,NULL,NULL,NULL,NULL,'dev_intermediate',NULL,NULL,'dev_curated','metrics_share',"with result as (
  select
    dd.before_or_after_5g,
    cities.city_name,
    dd.month_name,
    dd.time_period,
    metrix.*
  from
    {intermediate_schema}.fact_metrics_share as metrix
    join {intermediate_schema}.dim_date as dd on metrix.dated = dd.dated
    join {intermediate_schema}.dim_cities as cities on metrix.city_code = cities.city_code
)
merge into {curated_schema}.{curated_table} as target
  using result as source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.before_or_after_5g = source.before_or_after_5g,
      target.city_name = source.city_name,
      target.month_name = source.month_name,
      target.time_period = source.time_period,
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.company = source.company,
      target.company_revenue_crores = source.company_revenue_crores,
      target.arpu = source.arpu,
      target.active_users_lakhs = source.active_users_lakhs,
      target.unsubscribed_users_lakhs = source.unsubscribed_users_lakhs,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = '{LoadID}'
  when not matched then
    insert (
      target.before_or_after_5g,
      target.city_name,
      target.month_name,
      target.time_period,
      target.dated,
      target.city_code,
      target.company,
      target.company_revenue_crores,
      target.arpu,
      target.active_users_lakhs,
      target.unsubscribed_users_lakhs,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.before_or_after_5g,
      source.city_name,
      source.month_name,
      source.time_period,
      source.dated,
      source.city_code,
      source.company,
      source.company_revenue_crores,
      source.arpu,
      source.active_users_lakhs,
      source.unsubscribed_users_lakhs,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      \'{LoadID}\'
    )"),
    (208,NULL,NULL,NULL,NULL,'dev_intermediate',NULL,NULL,'dev_curated','market_share',"with result as (
  select
    dd.before_or_after_5g,
    cities.city_name,
    dd.month_name,
    dd.time_period,
    market.*
  from
    {intermediate_schema}.fact_market_share as market
    join {intermediate_schema}.dim_date as dd on market.dated = dd.dated
    join {intermediate_schema}.dim_cities as cities on market.city_code = cities.city_code
)
merge into {curated_schema}.{curated_table} as target
  using result as source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.before_or_after_5g = source.before_or_after_5g,
      target.city_name = source.city_name,
      target.month_name = source.month_name,
      target.time_period = source.time_period,
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.tmv_city_crores = source.tmv_city_crores,
      target.company = source.company,
      target.ms_pct = source.ms_pct,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = \'{LoadID}\'
  when not matched then
    insert (
      target.before_or_after_5g,
      target.city_name,
      target.month_name,
      target.time_period,
      target.dated,
      target.city_code,
      target.tmv_city_crores,
      target.company,
      target.ms_pct,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.before_or_after_5g,
      source.city_name,
      source.month_name,
      source.time_period,
      source.dated,
      source.city_code,
      source.tmv_city_crores,
      source.company,
      source.ms_pct,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      \'{LoadID}\'
    )"),
    (209,NULL,NULL,NULL,NULL,'dev_intermediate',NULL,NULL,'dev_curated','plan_revenue',"with result as (
  select
    dd.before_or_after_5g,
    cities.city_name,
    dd.month_name,
    dd.time_period,
    pl.plan_description,
    revenue.*
  from
    {intermediate_schema}.fact_plan_revenue as revenue
    join {intermediate_schema}.dim_date as dd on revenue.dated = dd.dated
    join {intermediate_schema}.dim_cities as cities on revenue.city_code = cities.city_code
    join {intermediate_schema}.dim_plan pl on revenue.plan = pl.plan
)
merge into {curated_schema}.{curated_table} as target
  using result as source
  on
    source.seq_no = target.seq_no
  When matched then
    Update set
      target.before_or_after_5g = source.before_or_after_5g,
      target.city_name = source.city_name,
      target.month_name = source.month_name,
      target.time_period = source.time_period,
      target.plan_description = source.plan_description,
      target.dated = source.dated,
      target.city_code = source.city_code,
      target.plan = source.plan,
      target.plan_revenue_crores = source.plan_revenue_crores,
      target.seq_no = source.seq_no,
      target.last_inserted_dttm_azure = source.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp = source.last_updated_dttm_gcp,
      target.load_id = \'{LoadID}\'
  when not matched then
    insert (
      target.before_or_after_5g,
      target.city_name,
      target.month_name,
      target.time_period,
      target.plan_description,
      target.dated,
      target.city_code,
      target.plan,
      target.plan_revenue_crores,
      target.seq_no,
      target.last_inserted_dttm_azure,
      target.last_updated_dttm_gcp,
      target.load_id
    )
    values (
      source.before_or_after_5g,
      source.city_name,
      source.month_name,
      source.time_period,
      source.plan_description,
      source.dated,
      source.city_code,
      source.plan,
      source.plan_revenue_crores,
      source.seq_no,
      source.last_inserted_dttm_azure,
      source.last_updated_dttm_gcp,
      '{LoadID}'
    )
")
""")